In [26]:
%load_ext autoreload
%autoreload 2



from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score
# noinspection PyUnresolvedReferences
from hcve_lib.tracking import load_run_results
from hcve_lib.utils import notebook_init, run_parallel
from hcve_lib.evaluation_functions import map_inverse_weight


notebook_init()


from notebooks.deps.binary_predictive_performance import run_roc_analysis, get_pr_analysis
from notebooks.deps.binary_predictive_performance import get_pr_analysis, get_pr_analysis_ci, plot_pr_ci

from mlflow import set_tracking_uri
from notebooks.deps.config import TIME_POINT_PREDICTION
from deps.common import get_data_cached

from sklearn.metrics import roc_curve
from pandas import DataFrame
from plotly.graph_objs import Figure

from deps.constants import RANDOM_STATE
from hcve_lib.evaluation_functions import average_group_scores, merge_standardize_prediction, merge_predictions, \
    compute_metrics_prediction
from hcve_lib.metrics import BootstrappedMetric
from hcve_lib.tracking import load_group_results
from hcve_lib.visualisation import setup_plotly_style
from hcve_lib.functional import t
import numpy as np
from hcve_lib.metrics import statistic_from_bootstrap
from hcve_lib.functional import reject_none
from plotly import express as px
from hcve_lib.utils import transpose_list
from notebooks.deps.binary_predictive_performance import run_pr_analysis_ci

from config import GROUPS_LCO, GROUPS_10_fold
from hcve_lib.metrics import BinaryMetricFromScore
from hcve_lib.data import binarize_event
from deps.data import load_raw_data, load_data


set_tracking_uri('http://localhost:5000')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data, metadata, X, y = get_data_cached()


[Memory]3.1s, 0.1min    : Loading get_data...
____________________________________________get_data cache loaded - 0.0s, 0.0min


In [28]:
raw_data = load_raw_data()

In [3]:
from hcve_lib.functional import dict_subset

GROUPS = dict_subset(['gb', 'coxnet', 'stacking', 'pcp_hf'], GROUPS_LCO)

In [4]:
TIME_POINT_PREDICTION = 10*365

In [5]:
binarized = binarize_event(TIME_POINT_PREDICTION, y['data'], drop_censored=False)

print('Before')
print(y['data']['label'].value_counts().to_dict())
print('↓')
print(f'Binarization {TIME_POINT_PREDICTION/365} years (NA removed)')
print((binarized.value_counts() - y['data']['label'].value_counts()).to_dict())
print('↓')
print(binarized.value_counts(dropna=True).to_dict())


Before
{0.0: 29286, 1.0: 1068}
↓
Binarization 10.0 years (NA removed)
{0.0: -27294, 1.0: -177}
↓
{0.0: 1992, 1.0: 891}


/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:247: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


In [6]:
from toolz import valmap 

merged_prediction = valmap(
    lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id))),
    GROUPS,
)


In [41]:
y_binarized = binarize_event(5*365, y['data'])

/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:247: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


In [65]:
y_proba_gb = merged_prediction['gb']['y_proba'][5*365].loc[X.index]
y_proba_coxnet = merged_prediction['coxnet']['y_proba'][5*365].loc[X.index]

In [66]:
manifold_X = X.to_dict('records')

In [71]:
X.to_csv('./output/X.csv')

In [73]:
DataFrame({'gb': 1-y_proba_gb, 'coxnet': 1-y_proba_coxnet}).to_csv('./output/prob.csv')

In [74]:
y_binarized.to_csv('./output/y_true.csv')